# Ejercicio

- Obtener de la API una lista de Tweets que no sean retweet y que contengan el hashtag #GRAMMYs en inglés. COMPLETADO
- Realizar la tokenización COMPLETADO
- Realizar un etiquetado POS con la función pos_tag de NLTK COMPLETADO
- Obtener la lista y frecuencia de los sustantivos en singular y plural# 



In [27]:

import os
from dotenv import load_dotenv
# Cargar valores del archivo .env en las variables de entorno
load_dotenv()
# Cargar valor del token a variable
bearer_token = os.environ.get("BEARER_TOKEN")


In [28]:
url = "https://api.twitter.com/2/tweets/search/recent"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "User-Agent":"v2FullArchiveSearchPython"
}
hastag='#GRAMMYs'
params = {
    'query': f'{hastag} -is:retweet lang:en', #Va a traer los que tienen el hashtag '#machinelearning' y el -is:retweet es para que no traigra retweets
    'max_results':100  #Y que traiga 100 resultados maximo
}

In [29]:
import requests
response = requests.get(url, headers=headers, params=params)
print(response)
# Generar excepción si la respuesta no es exitosa
if response.status_code != 200:
    raise Exception(response.status_code, response.text)
print(response.json())

<Response [200]>
{'data': [{'id': '1454523500153163779', 'text': 'We have many stories planned &amp; ready for you to read, but what celebrity would you like to see the most in our new magazine? #Hollywood #stars #celebrities #famous #glamour #event #redcarpet #spotlight #fashion #music #Awards #Oscars #GRAMMYs #actress #actor #movie #camera #film'}, {'id': '1454523344703918086', 'text': "#GRAMMYs Voting is critically important because the power, privilege, and prestige of recognizing music’s creators is in the hands of our @RecordingAcademy members. If you're a #RecordingAcademy voting member, don't forget to vote! #Vote4GRAMMYs https://t.co/Sm2lrL6O74"}, {'id': '1454522098173550607', 'text': 'For Your Grammy Consideration\n@RecordingAcad #GRAMMYs #grammys #useinbekirov https://t.co/nafBEkoCEL'}, {'id': '1454520926612062208', 'text': 'Are you a member of the Recording Academy or do you have work being considered for the 64th Annual GRAMMY® Awards?  \n\nI’m listening 🎧 to work in the c

In [30]:
import pandas as pd
df = pd.json_normalize(response.json()['data'])
df

,id,text
0,1454523500153163779,We have many stories planned &amp; ready for y...
1,1454523344703918086,#GRAMMYs Voting is critically important becaus...
2,1454522098173550607,For Your Grammy Consideration\n@RecordingAcad ...
3,1454520926612062208,Are you a member of the Recording Academy or d...
4,1454520817711255560,The latest Lady Starr Radio WSTR-DB Newsletter...
...,...,...
95,1454320070356713478,Word on the streets is that @JYPETWICE and #Bl...
96,1454317278250651649,"@bader_diedrich Well…Yeah…Cause, even when you..."
97,1454315063892979718,#GRAMMYs #grammyvote #lovease #soul #RnB #neos...
98,1454308282018017286,#GRAMMYs Voting is critically important becaus...


In [31]:
from nltk.tokenize import TweetTokenizer # libreria especifica para los tokenizar tweets 

tt = TweetTokenizer() # intanciamos ya que es un modelo lo que vamos a llamar
# Aplicar Tokenizer a la columna
tokenized_text = df['text'].apply(tt.tokenize)
df["tokenized_text"] = tokenized_text
df

,id,text,tokenized_text
0,1454523500153163779,We have many stories planned &amp; ready for y...,"[We, have, many, stories, planned, &, ready, f..."
1,1454523344703918086,#GRAMMYs Voting is critically important becaus...,"[#GRAMMYs, Voting, is, critically, important, ..."
2,1454522098173550607,For Your Grammy Consideration\n@RecordingAcad ...,"[For, Your, Grammy, Consideration, @RecordingA..."
3,1454520926612062208,Are you a member of the Recording Academy or d...,"[Are, you, a, member, of, the, Recording, Acad..."
4,1454520817711255560,The latest Lady Starr Radio WSTR-DB Newsletter...,"[The, latest, Lady, Starr, Radio, WSTR-DB, New..."
...,...,...,...
95,1454320070356713478,Word on the streets is that @JYPETWICE and #Bl...,"[Word, on, the, streets, is, that, @JYPETWICE,..."
96,1454317278250651649,"@bader_diedrich Well…Yeah…Cause, even when you...","[@bader_diedrich, Well, …, Yeah, …, Cause, ,, ..."
97,1454315063892979718,#GRAMMYs #grammyvote #lovease #soul #RnB #neos...,"[#GRAMMYs, #grammyvote, #lovease, #soul, #RnB,..."
98,1454308282018017286,#GRAMMYs Voting is critically important becaus...,"[#GRAMMYs, Voting, is, critically, important, ..."


In [32]:
import re
i=0
for text in df['tokenized_text']:
    #text=eval(str_text) # lo convertimos en lista para poder iterar por palabra
    #print("\nTEXTO A ANALIZAR ->",text)
    j=0
    for palabra in text:
        if(re.search('^#+',palabra)):
            palabra = re.sub(r'.', '', palabra, count = 1) # eliminamos el primer caracter que es "#"
            text[j]=palabra
            j=j+1
        elif(re.search('^@+',palabra)):
            palabra = re.sub(r'.', '', palabra, count = 1) # eliminamos el primer caracter que es "@"
            text[j]=palabra
            j=j+1
        elif(re.search('^http+',palabra)):
            text.remove(palabra)  #ACA NO SUMAMOS J PORQUE QUITAMOS UN ELEMENTO DE LA LISTA
        else:
            text[j]=palabra
            j=j+1
    #print("\n TEXTO DEVULETO ->", text) 
    df['tokenized_text'][i] = text
    i=i+1

df.to_csv('limpieza3')
df['tokenized_text']

['We',
 'have',
 'many',
 'stories',
 'planned',
 '&',
 'ready',
 'for',
 'you',
 'to',
 'read',
 ',',
 'but',
 'what',
 'celebrity',
 'would',
 'you',
 'like',
 'to',
 'see',
 'the',
 'most',
 'in',
 'our',
 'new',
 'magazine',
 '?',
 'Hollywood',
 'stars',
 'celebrities',
 'famous',
 'glamour',
 'event',
 'redcarpet',
 'spotlight',
 'fashion',
 'music',
 'Awards',
 'Oscars',
 'GRAMMYs',
 'actress',
 'actor',
 'movie',
 'camera',
 'film']

In [8]:

import nltk
#nltk.download('averaged_perceptron_tagger')
#nltk.download('tagsets')
df['pos_tokenized_text']=df['tokenized_text']

i=0
for text in df['tokenized_text']:    
    df['pos_tokenized_text'][i] = nltk.pos_tag(text)
    i=i+1
df['pos_tokenized_text'][0]

[('We', 'PRP'),
 ('have', 'VBP'),
 ('many', 'JJ'),
 ('stories', 'NNS'),
 ('planned', 'VBD'),
 ('&', 'CC'),
 ('ready', 'JJ'),
 ('for', 'IN'),
 ('you', 'PRP'),
 ('to', 'TO'),
 ('read', 'VB'),
 (',', ','),
 ('but', 'CC'),
 ('what', 'WP'),
 ('celebrity', 'NN'),
 ('would', 'MD'),
 ('you', 'PRP'),
 ('like', 'VB'),
 ('to', 'TO'),
 ('see', 'VB'),
 ('the', 'DT'),
 ('most', 'JJS'),
 ('in', 'IN'),
 ('our', 'PRP$'),
 ('new', 'JJ'),
 ('magazine', 'NN'),
 ('?', '.'),
 ('Hollywood', 'NNP'),
 ('stars', 'NNS'),
 ('celebrities', 'VBZ'),
 ('famous', 'JJ'),
 ('glamour', 'JJ'),
 ('event', 'NN'),
 ('redcarpet', 'NN'),
 ('spotlight', 'JJ'),
 ('fashion', 'NN'),
 ('music', 'NN'),
 ('Awards', 'NNP'),
 ('Oscars', 'NNP'),
 ('GRAMMYs', 'NNP'),
 ('actress', 'NN'),
 ('actor', 'NN'),
 ('movie', 'NN'),
 ('camera', 'NN'),
 ('film', 'NN')]

# Obtener la lista y frecuencia de los sustantivos en singular y plural# NN-> sustantivo NNS -> sustantivo en plural

In [9]:
df=df[['pos_tokenized_text']]
df

,pos_tokenized_text
0,"[(We, PRP), (have, VBP), (many, JJ), (stories,..."
1,"[(GRAMMYs, NNP), (Voting, NNP), (is, VBZ), (cr..."
2,"[(For, IN), (Your, PRP$), (Grammy, NNP), (Cons..."
3,"[(Are, NNP), (you, PRP), (a, DT), (member, NN)..."
4,"[(The, DT), (latest, JJS), (Lady, JJ), (Starr,..."
...,...
95,"[(Word, NNP), (on, IN), (the, DT), (streets, N..."
96,"[(bader_diedrich, RB), (Well, NNP), (…, NNP), ..."
97,"[(GRAMMYs, NNP), (grammyvote, NN), (lovease, N..."
98,"[(GRAMMYs, NNP), (Voting, NNP), (is, VBZ), (cr..."


In [10]:
df['pos_tokenized_text'][0][0][1]

'PRP'

In [11]:
from nltk.probability import FreqDist

tokenized_list = df.explode('pos_tokenized_text')
# Obtener frecuencia de cada término
list_nn_nns = [] ##creamos una lista donde vamos a meter las palabras
i=0
for text in df['pos_tokenized_text']:
    j=0
    for tupla in text:
        if (tupla[1]=='NN' or tupla[1]=='NNS'):
            print("->",tupla[0])
            list_nn_nns.append(tupla[0])
        j=j+1
    i=i+1


#fdist = FreqDist(tokenized_list['pos_tokenized_text'])

# Convertir a dataframe
#df_fdist = pd.DataFrame.from_dict(fdist, orient='index')
#df_fdist.columns = ['Frequency']
#df_fdist.index.name = 'Term'
#df_fdist.sort_values(by=['Frequency'], inplace=True)
#df_fdist

-> stories
-> celebrity
-> magazine
-> stars
-> event
-> redcarpet
-> fashion
-> music
-> actress
-> actor
-> movie
-> camera
-> film
-> power
-> privilege
-> prestige
-> music
-> s
-> creators
-> hands
-> members
-> voting
-> member
-> don't
-> forget
-> grammys
-> useinbekirov
-> member
-> work
-> 🎧
-> categories
-> Newsletter
-> samantha_azar
-> music
-> grammys
-> #grammys
-> co
-> listentojoshua
-> grammyconsideration
-> grammys21
-> b-side
-> news
-> bristol
-> theestallion
-> spotify
-> beyonce
-> halloween2021
-> grammyconsideration
-> grammy21
-> Today
-> missunderstood
-> queennaija
-> debut
-> debut
-> stream
-> category
-> album
-> queennaija
-> rnb
-> #GRAMMYs
-> power
-> privilege
-> prestige
-> music
-> s
-> creators
-> hands
-> members
-> voting
-> member
-> don't
-> forget
-> s
-> album
-> eligible
-> year
-> official
-> submissions
-> •
-> Year
-> Performance
-> version
-> thank
-> recordingacademy
-> billboard
-> mariachidivasdecindyshea
-> armandomanzanero
-> rigobe

In [12]:
print(list_nn_nns)

['stories', 'celebrity', 'magazine', 'stars', 'event', 'redcarpet', 'fashion', 'music', 'actress', 'actor', 'movie', 'camera', 'film', 'power', 'privilege', 'prestige', 'music', 's', 'creators', 'hands', 'members', 'voting', 'member', "don't", 'forget', 'grammys', 'useinbekirov', 'member', 'work', '🎧', 'categories', 'Newsletter', 'samantha_azar', 'music', 'grammys', '#grammys', 'co', 'listentojoshua', 'grammyconsideration', 'grammys21', 'b-side', 'news', 'bristol', 'theestallion', 'spotify', 'beyonce', 'halloween2021', 'grammyconsideration', 'grammy21', 'Today', 'missunderstood', 'queennaija', 'debut', 'debut', 'stream', 'category', 'album', 'queennaija', 'rnb', '#GRAMMYs', 'power', 'privilege', 'prestige', 'music', 's', 'creators', 'hands', 'members', 'voting', 'member', "don't", 'forget', 's', 'album', 'eligible', 'year', 'official', 'submissions', '•', 'Year', 'Performance', 'version', 'thank', 'recordingacademy', 'billboard', 'mariachidivasdecindyshea', 'armandomanzanero', 'rigober

In [13]:
fdist = FreqDist(list_nn_nns)

# Convertir a dataframe
df_fdist = pd.DataFrame.from_dict(fdist, orient='index')
df_fdist.columns = ['Frequency']
df_fdist.index.name = 'Term'
df_fdist.sort_values(by=['Frequency'], inplace=True)
df_fdist

,Frequency
Term,
stories,1
insta,1
month,1
blindnessawareness,1
nomination,1
...,...
rap,29
access,29
hiphop,30
